In [1]:
!pip install transformers tensorflow pandas scikit-learn --quiet

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import tensorflow as tf

In [2]:
df = pd.read_csv("cleaned_dataset.csv")
print( df.shape)

(1161, 14)


In [3]:
label_maping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
df['label'] = df['Sentiment'].map(label_maping)

In [4]:
train_x, test_x, train_y, test_y = train_test_split(
    df['Processed_Text'], df['label'], test_size=0.2, random_state=42
)

In [5]:
b_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

trained_token = b_tokenizer(
    list(train_x), truncation=True, padding=True, max_length=128, return_tensors="tf"
)
test_tokens = b_tokenizer(
    list(test_x), truncation=True, padding=True, max_length=128, return_tensors="tf"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


In [6]:
train_y = tf.convert_to_tensor(train_y.values)
test_y = tf.convert_to_tensor(test_y.values)

In [7]:
model = TFDistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=3,
    from_pt=True
)

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSeq

In [8]:
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(
    optimizer=adam_optimizer,
    loss=cross_entropy,
    metrics=["accuracy"]
)

In [9]:
training_history = model.fit(
    x={"input_ids": trained_token["input_ids"], "attention_mask": trained_token["attention_mask"]},
    y=train_y,
    batch_size=8,
    epochs=2,
    validation_split=0.1,
    verbose=2
)

Epoch 1/2
105/105 - 300s - loss: 0.2996 - accuracy: 0.9234 - val_loss: 0.0162 - val_accuracy: 1.0000 - 300s/epoch - 3s/step
Epoch 2/2
105/105 - 227s - loss: 0.0113 - accuracy: 1.0000 - val_loss: 0.0050 - val_accuracy: 1.0000 - 227s/epoch - 2s/step


In [10]:
results = model.predict({
    "input_ids": test_tokens["input_ids"],
    "attention_mask": test_tokens["attention_mask"]
})
predicted_classes = np.argmax(results.logits, axis=1)

print("\n Evaluation Results:")
print(classification_report(test_y, predicted_classes, target_names=['Negative', 'Neutral', 'Positive']))


8/8 [==============================] - 8s 615ms/step

 Evaluation Results:
              precision    recall  f1-score   support

    Negative       1.00      1.00      1.00        73
     Neutral       1.00      1.00      1.00        79
    Positive       1.00      1.00      1.00        81

    accuracy                           1.00       233
   macro avg       1.00      1.00      1.00       233
weighted avg       1.00      1.00      1.00       233



In [11]:
model.save_pretrained("sentiment_distilbert_model")
b_tokenizer.save_pretrained("sentiment_distilbert_model")

print("\n Model and tokenizer have been successfully saved.")



 Model and tokenizer have been successfully saved.
